<img src="images/dwave_leap.png" width="500 px" align="center">

# Getting Started with Ocean - Part 2





In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## Example: Friends & Enemies

<img src="images/fe_8.jpg" width="800 px" align="center">

Let us start with this slightly modified Friends and Enemy example. The respective QUBO is

$$
\rm{QUBO} = -2 +2x -2xy +2yz -2zx
$$

In [ ]:
import dimod 

# using the order: (x, y, z, w)
Q = {(0, 0): 2, (0, 1): -2, (1, 2): 2, (0, 2): -2}
Offset = -2

# construct the BQM
bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset=Offset)

In [ ]:
from utils.libqubo import show_bqm_graph
show_bqm_graph(bqm)

Now we need to get access to the QPU **solver** provided by D-Wave. A solver is a resource for solving problems. Some solvers interface to the QPU; others leverage CPU and GPU resources.

Solvers are responsible for:

- Encoding submitted problems
- Checking submitted parameters
- Decoding answers
- Adding problems to a client’s submission queue

With the solver as ressource we can use a **sampler** to obtain results for our problem. A sampler is a process that samples from low energy states of a problem’s objective function. A binary quadratic model (BQM) sampler samples from low energy states in models such as those defined by an Ising equation or a Quadratic Unconstrained Binary Optimization (QUBO) problem and returns an iterable of samples, in order of increasing energy. A dimod sampler provides ‘sample_qubo’ and ‘sample_ising’ methods as well as the generic BQM sampler method.

In [ ]:
from dwave.system.samplers import DWaveSampler  

MY_TOKEN = ""
# list the names of all solvers we can access
sampler = DWaveSampler(token=MY_TOKEN, solver={'qpu': True})
print('Successfully connected to D-Wave Sampler %s.' % sampler.client.get_solver().id)

In [ ]:
# obtain some information about the solver we use
info = sampler.solver.data.copy()
del info['properties']['qubits']
del info['properties']['couplers']
del info['properties']['anneal_offset_ranges']
info

Lets sample and obtain some results...

In [ ]:
# sample energy space
sampleset = sampler.sample(bqm, num_reads=1000)

Oh... obviously our problem can not be embedded on the hardware straight forward. This happens quite often and it is known as the so called embedding problem. So solve that we have to map multiple physical qubits to logic qubits just like in the example below. Multiple physical qubits can be combined to form a logical qubit as long as the form a subset on the Hardware graph, i.e. the form a chain.

<img src="images/embedding.png" width=1000>

So before we are able to encode our problem on a certain QPU we first need to find a suitable embedding. D-Wave provides different routines to find such embeddings, e.g. `minorminer.find_embedding`, `EmbeddingComposite`, `FixedEmbedding`, `LazyEmbedding`. For course you can also manually define your own embedding and hand it to the sampler.

In [ ]:
from minorminer import find_embedding
# get target structure
target_nodes, target_edges, _ = sampler.structure
# find an embedding suitable to the solver
embedding = find_embedding(Q, target_edges, verbose=2, random_seed=100)
print('\ninspect embedding:', embedding)

In [ ]:
from dwave.system.composites import FixedEmbeddingComposite, EmbeddingComposite, LazyEmbeddingComposite
# define embedded sampler

embedded_sampler = FixedEmbeddingComposite(sampler, embedding)
#embedded_sampler = LazyEmbeddingComposite(sampler)
#embedded_sampler = EmbeddingComposite(sampler)

In [ ]:
# sample energy space
numruns = 1000
sampleset = embedded_sampler.sample(bqm, num_reads=numruns, chain_strength=1)
print("QPU call complete using", sampleset.info['timing']['qpu_access_time']/1000000, "seconds of QPU time for", numruns, "samples.")

In [ ]:
# show samples
sampleset.to_pandas_dataframe()

# Workflow

1. Map your problem to a binary quadratic model (QUBO or ISING)
2. Select a solver
3. Find an Embedding of the BQM on the solver or use the D-Wave embeddings
4. Use a Sampler to sample the energy space of the BQM

# Questions:

- Whats the Embedding Problem?
- Whats the difference between a logical and a physical qubit?
- Which quantum effect is heavily made use of in a quantum annealer?
- Can any optimization problem be reformulated as a BQM?

# Exercise: 

Confirm the results from the previous Enemies & Friends problem on the QPU.